In [1]:
import gensim
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

# Sample corpus
corpus = [
    ['I', 'love', 'this', 'movie'],
    ['This', 'movie', 'is', 'terrible'],
    ['I', 'enjoy', 'watching', 'this', 'film'],
    ['This', 'film', 'is', 'fantastic'],
    ['I', 'hate', 'this', 'movie'],
    ['This', 'movie', 'is', 'great'],
    ['I', 'dislike', 'this', 'movie']
]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)

# Get embeddings for a word
print(word2vec_model.wv['movie'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[-5.3637347e-04  2.3664984e-04  5.1043299e-03  9.0108141e-03
 -9.3033137e-03 -7.1168668e-03  6.4601046e-03  8.9729978e-03
 -5.0165807e-03 -3.7636335e-03  7.3809074e-03 -1.5337755e-03
 -4.5362012e-03  6.5540806e-03 -4.8600291e-03 -1.8159966e-03
  2.8766592e-03  9.9195505e-04 -8.2863886e-03 -9.4502876e-03
  7.3122410e-03  5.0713350e-03  6.7588771e-03  7.6212117e-04
  6.3507855e-03 -3.4049307e-03 -9.4684598e-04  5.7689613e-03
 -7.5226836e-03 -3.9361427e-03 -7.5119403e-03 -9.3012175e-04
  9.5388936e-03 -7.3197382e-03 -2.3339323e-03 -1.9378333e-03
  8.0773048e-03 -5.9308102e-03  4.4817763e-05 -4.7542653e-03
 -9.6035041e-03  5.0074253e-03 -8.7603759e-03 -4.3916525e-03
 -3.4381239e-05 -2.9688881e-04 -7.6620355e-03  9.6152881e-03
  4.9828705e-03  9.2333183e-03 -8.1588905e-03  4.4960594e-03
 -4.1363789e-03  8.2365924e-04  8.4988978e-03 -4.4620461e-03
  4.5169741e-03 -6.7872386e-03 -3.5483269e-03  9.3989586e-03
 -1.5773958e-03  3.2078251e-04 -4.1405573e-03 -7.6825316e-03
 -1.5085457e-03  2.47032

In [2]:
# Sample sentiment data: (sentence, sentiment)
sentiment_data = [
    (['I', 'love', 'this', 'movie'], 'positive'),
    (['This', 'movie', 'is', 'terrible'], 'negative'),
    (['I', 'enjoy', 'watching', 'this', 'film'], 'positive'),
    (['This', 'film', 'is', 'fantastic'], 'positive'),
    (['I', 'hate', 'this', 'movie'], 'negative'),
    (['This', 'movie', 'is', 'great'], 'positive'),
    (['I', 'dislike', 'this', 'movie'], 'negative')
]


In [3]:
import numpy as np

# Convert words to embeddings
def word_to_embedding(word):
    if word in word2vec_model.wv:
        return word2vec_model.wv[word]
    else:
        return np.zeros(word2vec_model.vector_size)

# Prepare features and labels for the sentiment dataset
X = []
y = []
for sentence, sentiment in sentiment_data:
    sentence_embeddings = [word_to_embedding(word) for word in sentence]
    sentence_vector = np.mean(sentence_embeddings, axis=0)  # Average embeddings to get a fixed-size vector for the sentence
    X.append(sentence_vector)
    y.append(sentiment)

# Convert labels to numerical format
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
    positive       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
